# TD du 4 février 2025

# Exercice III

On considère le domaine $\Omega=[0,1]\times[0,1]$, la fonction $f\in L^{2}(\Omega)$, et l'EDP

$$(5.2)\quad\left\{
    \begin{array}{ll}
       - \Delta \displaystyle u=f &
       \textrm{dans}\ \Omega,\\
       u=0&\textrm{sur}\ \partial\Omega
    \end{array}\right.
$$

In [1]:
# On aura besoin des bibliothèques suivantes

import math
import numpy as np
import matplotlib.pyplot as plt
import sys
import time


# On aura également les besoins suivants 

from matplotlib import cm
from mpl_toolkits import mplot3d
from scipy import integrate

## Question 1

On souhaite écrire la formulation variationnelle. On suit les étapes indiquées dans l'exemple du handout https://cagnol.link/pde4handout

La formulation variationnelle obtenue est $$a(u,\varphi)=l(\varphi)$$
où $a$ est définie de $C^1_0(\Omega)\times C^1_0(\Omega)$ dans ${\mathbb{R}}$ par
$$a(u,v)=\int_{\Omega} \nabla u \cdot \nabla v$$
et $l$ est définie de $C^1_0(\Omega)$ dans $\mathbb {R}$ par $$l(v)=\int_{\Omega} gv$$
 


## Question 2

Définissons $L_x$ et $L_y$, posons $\Omega=[0,L_x]\times[0,L_y]$.

Choisissons la finesse de subdivision $J_x$ et $J_y$ de $[0,L_x]$ et $[0,L_y]$ respectivement. 

Pour $[0,L_x]$, on subdivise en $0,...,J_x+1$ points, il y en a donc $J_x+2$. 
Les points intérieurs (extrémités exclues) seront ainsi numérotés de 1 à $J_x$.

De manière analogue pour $[0,L_y]$, on subdivise en $0,...,J_y+1$ points, il y en a donc $J_y+2$.  Les points intérieurs (extrémités exclues) seront numérotées de 1 à $J_y$.

Comme on considère une subdivision uniforme, le pas (c'est-à-dire la longueur entre deux points de la discrétisation) est 
$$h_x = \frac{L_x}{J_x+1}$$
et de manière analogue
$$h_y = \frac{L_y}{J_y+1}$$


In [2]:
Lx = 6
Ly = 6

Jx = 5
Jy = 5

hx = Lx / (Jx+1)
hy = Ly / (Jy+1)

Soit $i\in\{1,...,J_y\}$ et $j\in\{1,...,J_x\}$.

Soit $(x,y)\in\Omega$, déterminons si $(x,y)$ dans l'hexagone ou non.
Si oui, déterminons dans quel triangle. La fonction suivante se charge de cela.

In [4]:
def QuelTriangle(x,y,j,i):

    if x>=j*hx and x<=(j+1)*hx and y>=i*hy and y<=(i+1)*hy: 
        # Quandrant nord-est
        
        if (x-j*hx)*hy/hx > (y-i*hy):            
            return 'T1'
        else: 
            return 'T2'
  

    elif x<=j*hx and x>=(j-1)*hx and y>=i*hy and y<=(i+1)*hy: 
        # Quandrant nord-ouest
        
        if y-hy*i <= (hy/hx)*(x-(j-1)*hx):
            return 'T3'
        else:
            return 'hors support'
 

    elif x<=j*hx and x>=(j-1)*hx and y<=i*hy and y>=(i-1)*hy: 
        # Quandrant sud-ouest
        
        if (x-j*hx)*hy/hx < (y-i*hy): 
            return 'T4'
        else:
            return 'T5'
        
        
    elif x>=j*hx and x<=(j+1)*hx and y<=i*hy and y>=(i-1)*hy: 
        # Quandrant sud-est
        
        if y>= hy/hx*x + hy*i-hy*(j+1):
            return 'T6'
        else:
            return 'hors support'
    
    else: 
        return 'hors support'

On détermine l'équation des six plans qui constituent les faces de la pyramide à six cotés, dont l'apex est en $(j h_x,i h_y,1)$. On en déduit l'expression de $\phi_{j,i}(x,y)$

In [5]:
def phi_ji(x,y,j,i):
    
    if x<0 or x>Lx or y<0 or y>Ly:
        return 0
    
    match QuelTriangle(x,y,j,i) :
        case 'T1':
            return -x/hx + j+1
        case 'T2':
            return -y/hy + i+1
        case 'T3':
            return x/hx - y/hy + i-j+1
        case 'T4':
            return x/hx + 1-j
        case 'T5':
            return y/hy + 1-i
        case 'T6':
            return -x/hx + y/hy + 1-i+j
        case 'hors support':
            return 0
    

SyntaxError: invalid syntax (3899112479.py, line 6)

In [6]:
print("Python version : ",sys.version)

Python version :  3.9.18 | packaged by conda-forge | (main, Dec 23 2023, 16:35:41) 
[Clang 16.0.6 ]


**Remarque** : si vous avez une erreur de syntaxe avec *match*, c'est par ce que votre version de Python est strictement inférieure à 3.10.  Dans ce cas, vous pouvez ré-écrire ce qui précède avec des if ... elif ... else.

In [7]:
def phi_ji(x,y,j,i):
    
    if x<0 or x>Lx or y<0 or y>Ly:
        return 0
    
    if   QuelTriangle(x,y,j,i) == "T1":
        return -x/hx + j+1
    elif QuelTriangle(x,y,j,i) == 'T2':
        return -y/hy + i+1
    elif QuelTriangle(x,y,j,i) == 'T3':
        return x/hx - y/hy + i-j+1
    elif QuelTriangle(x,y,j,i) == 'T4':
        return x/hx + 1-j
    elif QuelTriangle(x,y,j,i) == 'T5':
        return y/hy + 1-i
    elif QuelTriangle(x,y,j,i) == 'T6':
        return -x/hx + y/hy + 1-i+j
    elif QuelTriangle(x,y,j,i) == 'hors support':
        return 0